# Pivot tools for standard form linear-optimization problem $P$

## [Jon Lee](https://sites.google.com/site/jonleewebpage/home), 11/21/2020

For standard-form problems
\begin{align*}\tag{P}
z = \min~& c'x\\
&Ax   = b\\
&x \geq 0.
\end{align*}

Notes:
* Can work with $\epsilon$ perturbed right-hand side
* $\beta=(\beta_0,\beta_1,\ldots,\beta_{m-1})$ has $m$ entries from $\{0,1,\ldots,n-1\}$.
* $\eta=(\eta_0,\eta_1,\ldots,\eta_{n-m-1})$ has $n-m$ entries from $\{0,1,\ldots,n-1\}$.
* So, for the purpose of selecting $j$ (corresponding to $\eta_j$ entering the basis), we view $\bar c_{\eta}=(\bar c_{\eta_0},\bar c_{\eta_1},\ldots,\bar c_{\eta_{n-m-1}})$.
* For pivot_ratios(j): $j$ must be in $\{0,1,\ldots,n-m-1\}$. The output of pivot_ratios(j) is $m$ numbers, and they correspond to the basic variables numbered $\beta_0,\beta_1,\ldots,\beta_{m-1}$. So, for the purpose of selecting $i$ (correspond to $\beta_i$ leaving the basis), $i$ must be in $\{0,1,\ldots,m-1\}$.
* For pivot_swap(j,i): $j$ must be in $\{0,1,\ldots,n-m-1\}$ and $i$ must be in $\{0,1,\ldots,m-1\}$.

References:
* Jon Lee, "A First Course in Linear Optimization", Fourth Edition (Version 4.0), Reex Press, 2013-20.

MIT License

Copyright (c) 2020 Jon Lee

Permission is hereby granted, free of charge, to any person obtaining a copy 
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [ ]:
import numpy as np
import sympy as sym
sym.init_printing() 
import copy 
import operator
eps = sym.symbols('epsilon')
lam = sym.symbols('lambda')
xlz = sym.Symbol('\\bar{x}+\\lambda \\bar{z} :')
from IPython.display import Latex, Math
#########################################################################  
### CHOOSE A BACKEND --- IF YOU SWITCH, RESTART THE KERNEL
# evaluates faster than 'notebook': 
%matplotlib inline  
# evaluates slower than 'inline' (gives interactive plots, though delayed when running all cells):
#%matplotlib notebook  
######################################################################### 
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import itertools
import seaborn as sns; sns.set(); sns.set_style("whitegrid"); color_list = sns.color_palette("muted") 

In [ ]:
# perturb
def pivot_perturb():
    global m, b, Perturb, eps
    Perturb = True
    for i in range(m): 
        for j in range(m):
            b[i] += A_beta[i,j]*eps**(j+1)
    print('pivot_perturb() done')

In [ ]:
# algebra
def pivot_algebra():
    global m, n, objval, xbar, xbar_beta, xbar_eta, ybar, cbar_eta, ratios
    xbar_beta = A_beta.solve(b)
    xbar_eta = sym.zeros(n-m,1)
    objval = c_beta.dot(xbar_beta)
    xbar = sym.zeros(n,1)
    for i in range(m): xbar[beta[i]]=xbar_beta[i]
    for j in range(n-m): xbar[eta[j]]=xbar_eta[j]
    ybar = A_beta.transpose().solve(c[beta,0])
    #cbar_eta = c_eta.transpose()- ybar.transpose()*A_eta
    cbar_eta = c_eta- A_eta.transpose()*ybar
    ratios=sym.oo*sym.ones(m,1)
    print('pivot_algebra() done')

In [ ]:
# numerical version of a d-by-1 array
def N(parray):
    for i in range(parray.shape[0]): display(sym.N(parray[i]))

In [ ]:
# ratios (and direction) for a given nonbasic index eta_j
def pivot_ratios(j):
    global ratios, zbar
    if j>n-m-1:
        display(Latex("error: $j$ is out of range."))
    else:
        A_etaj=copy.copy(A[:,eta[j]])
        Abar_etaj = A_beta.solve(A_etaj)
        for i in range(m):
            if Abar_etaj[i]  > 0:
                ratios[i] = xbar_beta[i] / Abar_etaj[i] 
            else:
                ratios[i] = sym.oo
        display(ratios)
        zbar=sym.zeros(n,1)
        for i in range(m): zbar[beta[i]] = -Abar_etaj[i]
        zbar[eta[j]] = 1
        display(xlz,xbar+lam*zbar)

In [ ]:
# swap nonbasic eta_j in and basic beta_i out
def pivot_swap(j,i):
    global A_beta, A_eta, c_beta, c_eta
    if i>m-1 or j>n-m-1:
        display(Latex("error: $j$ or $i$ is out of range. swap not accepted"))
    else:
        save =  copy.copy(beta[i])
        beta[i] = copy.copy(eta[j])
        eta[j] = save
        A_beta = copy.copy(A[:,beta])
        A_eta = copy.copy(A[:,eta])
        c_beta = copy.copy(c[beta,0])
        c_eta  = copy.copy(c[eta,0])
        display(Latex("swap accepted --- new partition:"))
        print('eta:',eta)
        print('beta:',beta)
        print('*** MUST APPLY pivot_algebra()! ***')

In [ ]:
# plot
def pivot_plot():
    if n-m != 2 or Perturb == True:
        display(Latex("Hey friend --- give me a break!"))
        display(Latex("This plotting only works if there are $n-m=2$ nonbasic variables and no rhs perturbation"))
        return
    A_beta_inv = A_beta.inv()
    Abar_eta = A_beta_inv*A_eta
    M = sym.zeros(n,n-m)
    M[0:m,:] = Abar_eta
    M[m:n,:] = -sym.eye(n-m)
    h = sym.zeros(n,1)
    h[0:m,0] = xbar_beta
    feaspoints=np.empty((0,2))
    infeaspoints=np.empty((0,2))
    bbar=sym.zeros(2,1)
    M2=sym.zeros(2,2)
    for i in range(n-1):
        for j in range(i+1,n):
            bbar[0]=h[i]
            bbar[1]=h[j]
            M2[0,:]=M[i,:]
            M2[1,:]=M[j,:]
            if abs(sym.det(M2)) >0.0001:
                xy = M2.solve(bbar)
                if min(h - M*xy) >= -0.00001:
                    feaspoints=np.r_[feaspoints,np.transpose(xy)]
                else:
                    infeaspoints=np.r_[infeaspoints,np.transpose(xy)]
    hull = ConvexHull(feaspoints)
    fig, ax = plt.subplots(figsize=(8,8))
    ax.set(xlabel=r"$x_{}$".format(eta[0]), ylabel=r"$x_{}$".format(eta[1]))
    ax.spines['left'].set_position(('data',0.0))
    ax.spines['bottom'].set_position(('data',0.0))
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    plt.xlim(float(min(cbar_eta[0],min(feaspoints[:,0])))-1.25, float(max(feaspoints[:,0]))+0.25)
    plt.ylim(float(min(cbar_eta[1],min(feaspoints[:,1])))-0.25, float(max(feaspoints[:,1]))+0.25)
    plt.fill(feaspoints[hull.vertices,0], feaspoints[hull.vertices,1], 'cyan', alpha=0.3)
    x = np.linspace(float(min(feaspoints[:,0]))-0.5,float(max(feaspoints[:,0]))+0.5,100)
    for i in range(m):
        if Abar_eta[i,1] != 0:
            y = (xbar_beta[i] - Abar_eta[i,0]*x) / Abar_eta[i,1]
            plt.plot(x, y, linewidth=3, label=r"$x_{}$".format(beta[i]))
        else:
            plt.vlines(float(xbar_beta[i]/ Abar_eta[i,0]), float(min(cbar_eta[1],min(feaspoints[:,1]))), 
                       float(max(feaspoints[:,0])), label=r"$x_{}$".format(beta[i]))
    for simplex in hull.simplices:
         plt.fill(feaspoints[simplex, 0], feaspoints[simplex, 1], 'cyan', alpha=0.5)
    arrow=plt.arrow(0,0, float(cbar_eta[0]),float(cbar_eta[1]), color='magenta', width = 0.02, head_width = 0.1, label=r"$\bar{c}_\eta$")
    ax.scatter(feaspoints[:,0], feaspoints[:,1], color='green',zorder=8)
    ax.scatter(infeaspoints[:,0], infeaspoints[:,1], color='red',zorder=7)
    plt.legend(loc="upper left",title="slacks")
    plt.title(r"In the space of the non-basic variables",size=18)
    #ax.grid()
    plt.show()  

# Gomory cutting-plane tool for dual-form pure-integer problem $D_\mathcal{I}$

## [Jon Lee](https://sites.google.com/site/jonleewebpage/home), 11/21/2020

For dual-form pure-integer problem 
\begin{align*}\tag{$D_{\mathcal{I}}$}
\max~& y'b\\
&y'A   \leq c'\\
&y \in \mathbb{Z}^m.
\end{align*}

Notes:
* $A$ and $c$ ***MUST*** be integer
* The variables are $y_0,y_1,\ldots,y_{m-1}$, so valid input arguments for pure_gomory(i) are $i\in \{0,1,\ldots,m-1\}$.

Reference: 
* Qi He, Jon Lee. Another pedagogy for pure-integer Gomory. *RAIRO – Operations Research*, 51:189–197, 2017. 

In [ ]:
# pure gomory cut
def pure_gomory(i):
    global A, c, A_beta, A_eta, c_eta, cbar_eta, m, n, beta, eta
    if i>m-1:
        display(Latex("error: $i$ is out of range."))
    else:
        ei = sym.zeros(m,1)
        ei[i]=1                 # ei is the i-th standard unit column
        hi = A_beta.solve(ei)   # i-th column of basis inverse
        #r =  -sym.floor(hi)    # best choice of r
        r = -(hi.applyfunc(sym.floor))
        btilde = ei + A_beta*r  # new column for P
        A = A.row_join(btilde)
        c = c.col_join(sym.Matrix(([sym.floor(ybar.dot(btilde))])))
        eta.insert(n-m,n)
        n += 1
        A_eta = copy.copy(A[:,eta])
        c_eta  = copy.copy(c[eta,0])
        cbar_eta = c_eta - A_eta.transpose()*ybar
        print('*** PROBABLY WANT TO APPLY pivot_algebra()! ***')

In [ ]:
# dual plot
def dual_plot(delta=None,center=None):
    if delta==None: delta=2
    if center==None: center=ybar
    if m != 2  or Perturb == True:
        display(Latex("Hey friend --- give me a break!"))
        display(Latex("This plotting only works if there are $m=2$ dual variables and no rhs perturbation"))
        return
    M=sym.transpose(A)
    feaspoints=np.empty((0,2))
    infeaspoints=np.empty((0,2))
    c2=sym.zeros(2,1)
    M2=sym.zeros(2,2)
    for i in range(n-1):
        for j in range(i+1,n):
            c2[0]=c[i]
            c2[1]=c[j]
            M2[0,:]=M[i,:]
            M2[1,:]=M[j,:]
            if abs(sym.det(M2)) > 0.0001:
                y0y1 = M2.solve(c2)
                if min(c - M*y0y1) >= -0.00001:
                    feaspoints=np.r_[feaspoints,np.transpose(y0y1)]
                else:
                    infeaspoints=np.r_[infeaspoints,np.transpose(y0y1)]
    hull = ConvexHull(feaspoints)
    fig, ax = plt.subplots(figsize=(8,8))
    ax.xaxis.set_label_coords(1.05, 0.49)
    ax.yaxis.set_label_coords(0.5, 1.05)
    ax.set(xlabel=r"$y_{}$".format(0), ylabel=r"$y_{}$".format(1))
    ax.spines['left'].set_position(('data',ybar[0]))
    ax.spines['bottom'].set_position(('data',ybar[1]))
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    # set major ticks to show every 1 (integer)
    ax.xaxis.set_major_locator(MultipleLocator(1))
    ax.yaxis.set_major_locator(MultipleLocator(1))
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    plt.xlim(float(center[0])-delta,float(center[0])+delta)
    plt.ylim(float(center[1])-delta,float(center[1])+delta)
    plt.fill(feaspoints[hull.vertices,0], feaspoints[hull.vertices,1], 'cyan', alpha=0.3)
    y1 = np.linspace(float(min(feaspoints[:,0]))-0.5,float(max(feaspoints[:,0]))+0.5,100)     
    for j in range(n):
        if M[j,1] != 0:
            y2 = (c[j] - M[j,0]*y1) / M[j,1]
            plt.plot(y1, y2, linewidth=2, label=r"constraint ${}$".format(j))
        else:
            plt.vlines(float(c[j]/ M[j,0]), float(center[1])-delta, 
                       float(center[1])+delta, linewidth=2, label=r"constraint ${}$".format(j))
    for simplex in hull.simplices:
         plt.fill(feaspoints[simplex, 0], feaspoints[simplex, 1], 'cyan', alpha=0.5)
    arrow=plt.arrow(float(ybar[0]),float(ybar[1]),0.5*float(b[0]/(b.dot(b))**0.5),0.5*float(b[1]/(b.dot(b))**0.5), color='magenta', width = 0.01*delta, head_width = 0.02*delta, label=r"$b$")
    ax.scatter(feaspoints[:,0], feaspoints[:,1], color='green',zorder=8)
    ax.scatter(infeaspoints[:,0], infeaspoints[:,1], color='red',zorder=7)
    #  the integer grid 
    xp = np.arange(np.floor(float(center[0])-delta)-1, np.ceil(float(center[0])+delta)+2)
    yp = np.arange(np.floor(float(center[1])-delta)-1, np.ceil(float(center[1])+delta)+2)
    pp = itertools.product(xp, yp)
    plt.scatter(*zip(*pp), marker='o', s=5, color='black',zorder=9)
    #  sorting plot legend entries by label
    handles, labels = ax.get_legend_handles_labels()
    hl = sorted(zip(handles, labels), key=operator.itemgetter(1))
    handles2, labels2 = zip(*hl)
    ax.legend(handles2, labels2, loc="lower left",title="constraints")
    ax.grid(which='major')
    plt.show()  

In [ ]:
print('pivot_tools loaded: pivot_perturb, pivot_algebra, N, pivot_ratios, pivot_swap, pivot_plot, pure_gomory, mixed_gomory, dual_plot')